# Complex Networks 1st Project
Here we have the imports of and the graph being generated

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import collections
import numpy as np


# Read the graph
G = nx.read_graphml('airlines.graphml')



# https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.clustering.html
# https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.centrality.html



Printing various metrics

In [3]:
# How big the medium trip is
print("Average Path Length: ", nx.average_shortest_path_length(G))

# How popular are the airports that connect to the popular/unpopular airports
print("Average Degree Connectivity (mean k): ", nx.average_degree_connectivity(G))

# Average times that airports B<->C given that A<->C and A<->B
# print("Average Clustering: ", nx.average_clustering(G))

# Gives us how close each airport is to the others. Can compute the airport closest to every other in the world
print("Closeness Centrality: ", nx.closeness_centrality(G))

# The node that appears more times in the shortest paths between two other nodes
print("Closeness Centrality: ", nx.betweenness_centrality(G))

#TODO degree distribution histogram

Average Path Length:  2.3180578286961264
Average Degree Connectivity (mean k):  {16: 70.75, 6: 55.855555555555554, 3: 90.18518518518519, 13: 72.91346153846153, 4: 62.9921875, 9: 81.9047619047619, 27: 59.32098765432099, 22: 53.06818181818182, 28: 60.69047619047619, 5: 73.05, 7: 69.51785714285714, 8: 67.67045454545455, 2: 64.47142857142858, 23: 57.20652173913044, 18: 58.68888888888889, 10: 60.25555555555555, 20: 67.15, 15: 57.43333333333333, 25: 59.766666666666666, 11: 62.36363636363637, 135: 20.05925925925926, 30: 54.725, 36: 53.05555555555556, 248: 13.22983870967742, 14: 70.17857142857143, 29: 59.258620689655174, 144: 20.395833333333332, 197: 16.32994923857868, 94: 27.02127659574468, 17: 71.92436974789916, 57: 34.85087719298246, 12: 63.69444444444444, 19: 72.70175438596492, 33: 48.78787878787879, 37: 48.486486486486484, 38: 48.21052631578947, 24: 58.791666666666664, 172: 16.796511627906977, 21: 60.07142857142857, 258: 13.034883720930232, 32: 51.21875, 139: 20.928057553956833, 64: 24.35

<br>
Computing degree of the nodes and the respective number of nodes that have that degree
<br>

In [ ]:
# Get list with degrees of each node
degree_sequence = sorted([d for n, d in G.degree()], reverse=True)

print("###################################################################################")

# Dic with how much time each degree appeared
degreeCount = collections.Counter(degree_sequence)

# Separate dic into two separated tuples
deg, cnt = zip(*degreeCount.items())
cnt = list(cnt)


# Normalize cnt getting %
total_sum = sum(cnt)
for i in range(len(cnt)):
    cnt[i] = cnt[i]/total_sum

# Plotting the power law without any scale transformation

In [ ]:

ax = plt.subplot()
plt.bar(deg, cnt, width=0.5, color='b', tick_label=[d for d in deg])
plt.plot(deg, cnt, linestyle='--')
plt.title("Degree Distribution")
plt.ylabel("Probability P(k)")
plt.xlabel("Degree k")

plt.xticks([258])

plt.show()

# Transforming the scale to a log log scale

<br>
Plotting the Power Law
<br>
And the respective Cummulative distributed function

In [ ]:
###################################################################################
#Plotting the power law in a log log scale ---- Non cummulative plot
###################################################################################

plt.figure()
plt.title("Degree Distribution (log/log)")
plt.ylabel("log₁₀ P(k)")
plt.xlabel("log₁₀ k")

plt.loglog(deg, cnt, 'o')


#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------


###################################################################################
#Plotting the power law in a log log scale ---- Cummulative plot
###################################################################################


# Build cummulative vector
cnt_cdf = []
for i in range(len(cnt)):
	cnt_cdf += [sum(cnt[0:i+1])]

plt.figure()

plt.title("Cumulative Degree Distribution (log/log)")
plt.ylabel("log₁₀ P_cum(k)")
plt.xlabel("log₁₀ k")
plt.loglog(deg, cnt_cdf, 'o')

plt.show()

#package python to compute the linear line is named power law https://pypi.org/project/powerlaw/

In [ ]:
import powerlaw as pl


fit = pl.Fit(cnt_cdf)


print("Alpha value of network: ", fit.alpha)

In [ ]:
fit.plot_ccdf(color = 'b', marker='o', linewidth= 2)

In [ ]:
fig1 = fit.plot_ccdf(color = 'b', marker='o', linewidth= 2)
fit.power_law.plot_ccdf(color='b', linestyle='--', ax=fig1)
